# Dydra API client

> Tools to interact with the Dydra API

In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import rdflib
from dotenv import load_dotenv
import os
from SPARQLWrapper import SPARQLWrapper, JSON
import shutil
from tqdm import tqdm
import tempfile
from rdflib import Graph

In [ ]:
#| export
class DydraClient():    
    """
    A client for interacting with a Dydra SPARQL endpoint.

    Attributes:
        endpoint (str): The SPARQL endpoint URL.
        api_key (str): The API key for authorization.
    """

    @staticmethod
    def load_env(path):
        """
        Loads environment variables from a specified .env file.
        
        Args:
            path (str): Path to the .env file.
            
        Returns:
            tuple: Contains the endpoint and API key as strings.
        """

        load_dotenv(path)

        return os.getenv("DYDRA_ENDPOINT"), os.getenv("DYDRA_API_KEY")


    def __init__(self, endpoint, api_key):
        """
        Initializes the DydraClient with the given endpoint and API key.
        
        Args:
            endpoint (str): The SPARQL endpoint URL.
            api_key (str): The API key for authorization.
        """
        self.endpoint = endpoint
        self.api_key = api_key

    def import_by_file(self, file_path, format, graph_uri=None, verbose=False):
        """
        Imports RDF data from a file into the Dydra store.

        Args:
            file_path (str): The path to the RDF file to import.
            format (str): The format of the RDF file (e.g., 'xml', 'nt').
            graph_uri (str, optional): URI of the graph where data will be inserted. Defaults to None.
        """

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/sparql-update"
        }

        files = self._chunk_rdf_file(file_path, format=format)

        print("Number of chunks: ", len(files))

        for file in tqdm(files):

            # RDFファイルの読み込み
            graph = rdflib.Graph()
            graph.parse(file, format=format)  # フォーマットはファイルに応じて変更

            nt_data = graph.serialize(format='nt')

            if graph_uri is None:
                query = f"""
                INSERT DATA {{
                {nt_data}
                }}
                """

            else:
                query = f"""
                INSERT DATA {{
                GRAPH <{graph_uri}> {{
                    {nt_data}
                }}
                }}
                """

            if verbose:
                print(query)

            response = requests.post(self.endpoint, data=query, headers=headers)
            if response.status_code == 200:
                print("Data successfully inserted.")
            else:
                print(f"Error: {response.status_code} {response.text}")

    def delete_by_file(self, file_path, format, graph_uri=None, verbose=False):
        """
        Deletes RDF data from a file in the Dydra store.

        Args:
            file_path (str): The path to the RDF file to delete.
            format (str): The format of the RDF file (e.g., 'xml', 'nt').
            graph_uri (str, optional): URI of the graph where data will be deleted. Defaults to None.
        """

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/sparql-update"
        }

        files = self._chunk_rdf_file(file_path, format=format)

        print("Number of chunks: ", len(files))

        for file in tqdm(files):

            # RDFファイルの読み込み
            graph = rdflib.Graph()
            graph.parse(file, format=format)  # フォーマットはファイルに応じて変更

            nt_data = graph.serialize(format='nt')

            if graph_uri is None:
                query = f"""
                DELETE DATA {{
                {nt_data}
                }}
                """

            else:

                query = f"""
                DELETE DATA {{
                GRAPH <{graph_uri}> {{
                    {nt_data}
                }}
                }}
                """

            if verbose:
                print(query)

            response = requests.post(self.endpoint, data=query, headers=headers)
            if response.status_code == 200:
                print("Data successfully deleted.")
            else:
                print(f"Error: {response.status_code} {response.text}")

    def query(self, query):
        """
        Executes a SPARQL query against the endpoint and returns the results.

        Args:
            query (str): The SPARQL query to execute.

        Returns:
            dict: The query results in JSON format.
        """

        # SPARQLエンドポイントの設定
        sparql = SPARQLWrapper(self.endpoint)

        # クエリの設定
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)

        results = sparql.query().convert()

        return results
    
    def clear(self, graph_uri=None):
        """
        Clears all data from the default or specified graph.

        Args:
            graph_uri (str, optional): URI of the graph to clear. Defaults to None.
        """

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/sparql-update"
        }

        if graph_uri is None:
            query = """
            DELETE {
            ?s ?p ?o
            }
            WHERE {
            ?s ?p ?o
            }
            """

        else:
            query = f"""
            DELETE {{
            GRAPH <{graph_uri}> {{
                ?s ?p ?o
            }}
            }}
            WHERE {{
            GRAPH <{graph_uri}> {{
                ?s ?p ?o
            }}
            }}
            """

        response = requests.post(self.endpoint, data=query, headers=headers)

        if response.status_code == 200:
            print("Data successfully cleared.")
        else:
            print(f"Error: {response.status_code} {response.text}")

    def _serialize_chunk(self, graph, index, format='turtle'):
        output_dir = self.output_dir
        """チャンクをファイルにシリアライズする補助関数"""
        filename = f"{output_dir}/output_chunk_{index}.{format}"
        os.makedirs(output_dir, exist_ok=True)
        graph.serialize(filename, format=format)
        # print(f"Saved {filename}, size: {os.path.getsize(filename)} bytes")

        return filename

    def _clear_output_dir(self, output_dir):
        shutil.rmtree(output_dir, ignore_errors=True)

    def _chunk_rdf_file(self, input_file, output_dir=None, chunk_size=500*1024, format='turtle'):
        if output_dir is None:
            output_dir = tempfile.mkdtemp()

        self.output_dir = output_dir

        self._clear_output_dir(output_dir)

        """RDF ファイルを指定されたサイズのチャンクに分割する"""
        g = Graph()
        g.parse(input_file, format=format)

        current_chunk = Graph()
        current_size = 0
        chunk_index = 1

        count = 0

        files = []

        print("Triple count: ", len(g))

        for s, p, o in tqdm(g):
            count += 1

            # 一時的なグラフにトリプルを追加
            current_chunk.add((s, p, o))

            if count % 1000 == 0:

                # シリアライズしてサイズを確認
                temp_serialized = current_chunk.serialize(format=format)
                temp_size = len(temp_serialized)

                # 現在のチャンクサイズが設定値を超えたらシリアライズしてファイルに保存
                if temp_size >= chunk_size:
                    path = self._serialize_chunk(current_chunk, chunk_index, format=format)
                    files.append(path)
                    current_chunk = Graph()  # 新しいグラフを開始
                    chunk_index += 1

        # 最後のチャンクを保存
        if len(current_chunk) > 0:
            path = self._serialize_chunk(current_chunk, chunk_index, format=format)
            files.append(path)

        return files

    def delete_by_subjects(self, subject_uris, graph_uri=None, verbose=False):
        """
        Deletes RDF data from the Dydra store based on a list of subject URIs.

        Args:
            subject_uris (list): List of subject URIs to delete.
        """

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/sparql-update"
        }

        # Function to chunk URIs into batches
        def _chunked_uris(uris, chunk_size):
            for i in range(0, len(uris), chunk_size):
                yield uris[i:i + chunk_size]

        # Iterate over URI chunks
        for uri_chunk in tqdm(_chunked_uris(subject_uris, 100)):
            if graph_uri is None:
                query = f"""
                DELETE {{
                    ?uri ?p ?o
                }}
                WHERE {{
                    VALUES ?uri {{ {' '.join(f'<{uri}>' for uri in uri_chunk)} }}
                    ?uri ?p ?o
                }}
                """
            else:
                query = f"""
                DELETE {{
                    GRAPH <{graph_uri}> {{
                        ?uri ?p ?o
                    }}
                }}
                WHERE {{
                    GRAPH <{graph_uri}> {{
                        VALUES ?uri {{ {' '.join(f'<{uri}>' for uri in uri_chunk)} }}
                        ?uri ?p ?o
                    }}
                }}
                """

            if verbose:
                # print(query)
                pass

            response = requests.post(self.endpoint, data=query, headers=headers)
            if response.status_code == 200:
                if verbose:
                    print(f"Data for URIs successfully deleted.")
            else:
                print(f"Error: {response.status_code} {response.text}")

        '''
        for uri in tqdm(subject_uris):

            if graph_uri is None:

                query = f"""
            DELETE {{
            <{uri}> ?p ?o
            }}
            WHERE {{
            <{uri}> ?p ?o
            }}
            """
                
            else:
                query = f"""
            DELETE {{
            GRAPH <{graph_uri}> {{
                <{uri}> ?p ?o
            }}
            }}
            WHERE {{
            GRAPH <{graph_uri}> {{
                <{uri}> ?p ?o
            }}
            }}
            """
                
            if verbose:
                print(query)

            response = requests.post(self.endpoint, data=query, headers=headers)
            if response.status_code == 200:
                # print(f"Data for {uri} successfully deleted.")
                pass
            else:
                print(f"Error: {response.status_code} {response.text}")
        '''


In [ ]:
show_doc(DydraClient.import_by_file)

---

[source](https://github.com/nakamura196/dydra-py/blob/main/dydra_py/core.py#L29){target="_blank" style="float:right; font-size:smaller"}

### DydraClient.import_by_file

>      DydraClient.import_by_file (file_path, format, graph_uri=None,
>                                  verbose=False)

*Imports RDF data from a file into the Dydra store.

Args:
    file_path (str): The path to the RDF file to import.
    format (str): The format of the RDF file (e.g., 'xml', 'nt').
    graph_uri (str, optional): URI of the graph where data will be inserted. Defaults to None.*

In [ ]:
show_doc(DydraClient.delete_by_file)

---

[source](https://github.com/nakamura196/dydra-py/blob/main/dydra_py/api.py#L107){target="_blank" style="float:right; font-size:smaller"}

### DydraClient.delete_by_file

>      DydraClient.delete_by_file (file_path, format, graph_uri=None,
>                                  verbose=False)

*Deletes RDF data from a file in the Dydra store.

Args:
    file_path (str): The path to the RDF file to delete.
    format (str): The format of the RDF file (e.g., 'xml', 'nt').
    graph_uri (str, optional): URI of the graph where data will be deleted. Defaults to None.*

In [ ]:
show_doc(DydraClient.delete_by_subjects)

---

[source](https://github.com/nakamura196/dydra-py/blob/main/dydra_py/api.py#L287){target="_blank" style="float:right; font-size:smaller"}

### DydraClient.delete_by_subjects

>      DydraClient.delete_by_subjects (subject_uris, graph_uri=None,
>                                      verbose=False)

*Deletes RDF data from the Dydra store based on a list of subject URIs.

Args:
    subject_uris (list): List of subject URIs to delete.*

In [ ]:
show_doc(DydraClient.query)

---

[source](https://github.com/nakamura196/dydra-py/blob/main/dydra_py/core.py#L66){target="_blank" style="float:right; font-size:smaller"}

### DydraClient.query

>      DydraClient.query (query)

*Executes a SPARQL query against the endpoint and returns the results.

Args:
    query (str): The SPARQL query to execute.

Returns:
    dict: The query results in JSON format.*

In [ ]:
show_doc(DydraClient.clear)

---

[source](https://github.com/nakamura196/dydra-py/blob/main/dydra_py/core.py#L82){target="_blank" style="float:right; font-size:smaller"}

### DydraClient.clear

>      DydraClient.clear (graph_uri=None)

*Clears all data from the default or specified graph.

Args:
    graph_uri (str, optional): URI of the graph to clear. Defaults to None.*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()